In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import env
import scipy.stats as stats
import numpy as np
from sklearn.impute import SimpleImputer
import get_function as gf

**Aquire**

In [2]:
df_zillow = gf.get_zillow_data()

In [3]:
df_zillow.head()

,parcelid,logerror,transactiondate,id,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,12177905,-0.103410,2017-01-01,2288172,12177905,NaN,None,None,3.0,4.0,...,NaN,None,108918.0,145143.0,2016.0,36225.0,1777.51,None,NaN,6.037300e+13
1,12095076,-0.001011,2017-01-01,781532,12095076,1.0,None,None,3.0,4.0,...,NaN,None,276684.0,773303.0,2016.0,496619.0,9516.26,None,NaN,6.037461e+13
2,12069064,0.101723,2017-01-01,870991,12069064,NaN,None,None,1.0,2.0,...,NaN,None,18890.0,218552.0,2016.0,199662.0,2366.08,None,NaN,6.037302e+13
3,12790562,-0.040966,2017-01-02,1246926,12790562,NaN,None,None,3.0,4.0,...,NaN,None,177527.0,220583.0,2016.0,43056.0,3104.19,None,NaN,6.037500e+13
4,11104527,0.005963,2017-01-02,1639362,11104527,1.0,None,None,3.0,4.0,...,NaN,None,271715.0,430108.0,2016.0,158393.0,6103.36,None,NaN,6.037920e+13


In [4]:
df_zillow.shape

(33701, 62)

**Prepare**

In [6]:
df_trim = gf.handle_missing_values(df_zillow, prop_required_column = .6, prop_required_row = .75)

In [7]:
df_trim.isnull().sum()

parcelid                          0
logerror                          0
transactiondate                   0
id                                0
parcelid                          0
bathroomcnt                       0
bedroomcnt                        0
buildingqualitytypeid           127
calculatedbathnbr                 0
calculatedfinishedsquarefeet      0
finishedsquarefeet12              0
fips                              0
fullbathcnt                       0
heatingorsystemtypeid           536
latitude                          0
longitude                         0
lotsizesquarefeet               304
propertycountylandusecode         0
propertylandusetypeid             0
propertyzoningdesc               68
rawcensustractandblock            0
regionidcity                    694
regionidcounty                    0
regionidzip                      12
roomcnt                           0
unitcnt                           0
yearbuilt                         7
structuretaxvaluedollarcnt  

In [8]:
df_trim.shape

(33701, 33)

In [9]:
df_trim = gf.remove_columns(df_trim, ['roomcnt','assessmentyear','unitcnt','regionidzip','regionidcounty','propertylandusetypeid','propertycountylandusecode','fullbathcnt','calculatedbathnbr','rawcensustractandblock','finishedsquarefeet12','parcelid','transactiondate','id','regionidcity','buildingqualitytypeid','heatingorsystemtypeid','propertyzoningdesc','censustractandblock'])

In [10]:
df_trim.isnull().sum()

logerror                          0
bathroomcnt                       0
bedroomcnt                        0
calculatedfinishedsquarefeet      0
fips                              0
latitude                          0
longitude                         0
lotsizesquarefeet               304
yearbuilt                         7
structuretaxvaluedollarcnt       60
taxvaluedollarcnt                 0
landtaxvaluedollarcnt             0
taxamount                         4
dtype: int64

In [11]:
df_trim.shape

(33701, 13)

In [12]:
df_imputed = gf.imputer_9000(df_trim,['lotsizesquarefeet','structuretaxvaluedollarcnt','taxamount'],'mean')

In [13]:
df_imputed.isnull().sum()

logerror                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
yearbuilt                       7
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
dtype: int64

In [14]:
df_imputed = gf.imputer_9000(df_trim,['yearbuilt'],'most_frequent')

In [15]:
df_imputed.isnull().sum()

logerror                        0
bathroomcnt                     0
bedroomcnt                      0
calculatedfinishedsquarefeet    0
fips                            0
latitude                        0
longitude                       0
lotsizesquarefeet               0
yearbuilt                       0
structuretaxvaluedollarcnt      0
taxvaluedollarcnt               0
landtaxvaluedollarcnt           0
taxamount                       0
dtype: int64

In [16]:
df_imputed.head(25).T

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
logerror,-1.034097e-01,-1.010962e-03,1.017233e-01,-4.096564e-02,5.963169e-03,-3.508459e-02,-1.209759e-01,4.204126e-02,2.011018e-02,2.064699e-01,...,1.090182e-02,-1.347616e-01,7.515622e-02,-1.428001e-02,-2.594477e-01,1.970075e-02,-6.958367e-02,1.029678e-01,-2.871869e-02,-4.975211e-02
bathroomcnt,3.000000e+00,3.000000e+00,1.000000e+00,3.000000e+00,3.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,2.000000e+00,2.000000e+00,...,3.000000e+00,2.000000e+00,2.000000e+00,1.000000e+00,5.000000e+00,3.000000e+00,2.000000e+00,1.000000e+00,3.000000e+00,2.000000e+00
bedroomcnt,4.000000e+00,4.000000e+00,2.000000e+00,4.000000e+00,4.000000e+00,2.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,3.000000e+00,...,4.000000e+00,2.000000e+00,3.000000e+00,3.000000e+00,4.000000e+00,4.000000e+00,3.000000e+00,2.000000e+00,3.000000e+00,2.000000e+00
calculatedfinishedsquarefeet,2.376000e+03,2.962000e+03,7.380000e+02,3.039000e+03,2.540000e+03,1.146000e+03,1.316000e+03,1.641000e+03,1.538000e+03,1.851000e+03,...,3.160000e+03,1.529000e+03,1.447000e+03,8.720000e+02,5.492000e+03,2.317000e+03,1.815000e+03,6.090000e+02,2.380000e+03,1.814000e+03
fips,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,...,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03,6.037000e+03
latitude,3.424518e+07,3.414520e+07,3.414921e+07,3.396023e+07,3.443497e+07,3.402404e+07,3.400947e+07,3.376095e+07,3.437514e+07,3.410337e+07,...,3.415584e+07,3.404320e+07,3.416637e+07,3.395505e+07,3.414460e+07,3.424719e+07,3.420511e+07,3.411043e+07,3.410478e+07,3.383320e+07
longitude,-1.182407e+08,-1.181798e+08,-1.182394e+08,-1.180069e+08,-1.184644e+08,-1.181591e+08,-1.182811e+08,-1.181236e+08,-1.185598e+08,-1.182933e+08,...,-1.183756e+08,-1.183556e+08,-1.181513e+08,-1.183443e+08,-1.178185e+08,-1.184433e+08,-1.184540e+08,-1.179198e+08,-1.180563e+08,-1.181915e+08
lotsizesquarefeet,1.303800e+04,6.300000e+04,4.214000e+03,2.002800e+04,1.038400e+04,4.657000e+03,5.091000e+03,5.302000e+03,5.997000e+03,6.714000e+03,...,1.650600e+04,3.640000e+03,7.283000e+03,5.566000e+03,6.731600e+04,8.703000e+03,1.031100e+04,8.997000e+03,9.960000e+03,5.372000e+03
yearbuilt,1.970000e+03,1.950000e+03,1.922000e+03,1.970000e+03,1.999000e+03,1.939000e+03,1.908000e+03,1.944000e+03,1.972000e+03,1.920000e+03,...,1.941000e+03,1.928000e+03,1.901000e+03,1.922000e+03,2.006000e+03,1.953000e+03,1.958000e+03,1.948000e+03,1.991000e+03,1.962000e+03
structuretaxvaluedollarcnt,1.089180e+05,2.766840e+05,1.889000e+04,1.775270e+05,2.717150e+05,1.279600e+05,3.766300e+04,9.803800e+04,2.222380e+05,3.226400e+04,...,2.844980e+05,1.676330e+05,5.773400e+04,6.298600e+04,1.177804e+06,1.241400e+05,6.065600e+04,5.617100e+04,3.555210e+05,1.646000e+05


In [29]:
df_clean = df_imputed.rename(columns={'lotsizesquarefeet':'lot_square_feet','calculatedfinishedsquarefeet':'home_square_feet', 'logerror': 'target','bathroomcnt': 'bathroom', 'bedroomcnt': 'bedroom','structuretaxvaluedollarcnt':'structure_value','taxvaluedollarcnt': 'total_value','landtaxvaluedollarcnt':'land_value','taxamoun':'tax_amount','taxamount':'tax_amount','yearbuilt':'year_built'})

In [30]:
df_clean.head()

,target,bathroom,bedroom,home_square_feet,fips,latitude,longitude,lot_square_feet,year_built,structure_value,total_value,land_value,tax_amount
0,-0.103410,3.0,4.0,2376.0,6037.0,34245180.0,-118240722.0,13038.0,1970.0,108918.0,145143.0,36225.0,1777.51
1,-0.001011,3.0,4.0,2962.0,6037.0,34145202.0,-118179824.0,63000.0,1950.0,276684.0,773303.0,496619.0,9516.26
2,0.101723,1.0,2.0,738.0,6037.0,34149214.0,-118239357.0,4214.0,1922.0,18890.0,218552.0,199662.0,2366.08
3,-0.040966,3.0,4.0,3039.0,6037.0,33960230.0,-118006914.0,20028.0,1970.0,177527.0,220583.0,43056.0,3104.19
4,0.005963,3.0,4.0,2540.0,6037.0,34434971.0,-118464390.0,10384.0,1999.0,271715.0,430108.0,158393.0,6103.36


In [34]:
df_clean.target.max()

5.2629986498

In [35]:
df_clean.target.min()

-4.65542037764